In [154]:
import json, folium
import pandas as pd
import numpy as np

In [155]:
filename = 'data/공원.csv'
df = pd.read_csv(filename, encoding='utf-8',skiprows=2)
df.head()

,자치구별(1),자치구별(2),면적 (천㎡),1인당 공원면적 (㎡),면적 (천㎡).1,1인당 공원면적 (㎡).1,면적 (천㎡).2,1인당 공원면적 (㎡).2
0,합계,소계,172675.05,17.74,119276.76,12.25,54998.00,5.65
1,합계,종로구,11404.09,74.15,6282.46,40.85,3026.45,19.68
2,합계,중구,3178.19,24.12,3090.87,23.45,1359.55,10.32
3,합계,용산구,1775.55,7.48,759.11,3.20,1612.66,6.80
4,합계,성동구,3074.32,10.50,1231.01,4.21,2896.42,9.90


In [156]:
cols = '자치구별(2),면적 (천㎡),1인당 공원면적 (㎡)'.split(',')
df = df[cols]
df = df.rename(columns={'자치구별(2)': '구별','면적 (천㎡)':'면적','1인당 공원면적 (㎡)':'인당공원면적'})
df.drop([df.index[0]],inplace=True)
df.drop([df.index[-1]],inplace=True)
df.head()

,구별,면적,인당공원면적
1,종로구,11404.09,74.15
2,중구,3178.19,24.12
3,용산구,1775.55,7.48
4,성동구,3074.32,10.50
5,광진구,3459.84,9.81


In [157]:
df.tail()

,구별,면적,인당공원면적
21,관악구,12484.99,25.00
22,서초구,15072.03,36.22
23,강남구,7791.35,14.49
24,송파구,4771.42,7.19
25,강동구,3407.66,7.31


In [158]:
type(df.iloc[2,1]),type(df.iloc[2,2])

(numpy.float64, str)

In [159]:
df.면적 = (df.면적 *1000).apply(lambda x:float(x))
df.tail()

,구별,면적,인당공원면적
21,관악구,12484990.0,25.00
22,서초구,15072030.0,36.22
23,강남구,7791350.0,14.49
24,송파구,4771420.0,7.19
25,강동구,3407660.0,7.31


In [160]:
df['인당공원면적'] = df['인당공원면적'].apply(lambda x:float(x))
df.tail()

,구별,면적,인당공원면적
21,관악구,12484990.0,25.00
22,서초구,15072030.0,36.22
23,강남구,7791350.0,14.49
24,송파구,4771420.0,7.19
25,강동구,3407660.0,7.31


In [161]:
df.면적 = df.면적.astype(int)
df['인당공원면적']  = df['인당공원면적'].astype(int)
df.tail()

,구별,면적,인당공원면적
21,관악구,12484990,25
22,서초구,15072030,36
23,강남구,7791350,14
24,송파구,4771420,7
25,강동구,3407660,7


In [162]:
type(df.면적[2])

numpy.int32

In [163]:
file = '../04.지도시각화/data/seoul_geo_simple.json'
geo_str = json.load(open(file, encoding='utf-8'))

In [164]:
def get_text_location(ger_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']]= [np.mean(geo[:,1]),np.mean(geo[:,0])]
    return gu_dict
gu_dict = get_text_location(geo_str)

In [165]:
park_map = folium.Map(location=[37.55,126.98], zoom_start=11,tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=df.인당공원면적,           # 단계구분도로 보여줄 데이터
    columns=[df.index,df.인당공원면적],      # df에서 추출할 항목
    fill_color='PuRd',                  # Colormap
    key_on='feature.id'                # 지도에서 조인할 항목
).add_to(park_map)
park_map

In [166]:
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(park_map)
park_map

KeyError: 1